In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,  # featureScaling
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#  image argumentation : 과적합을 방지하기 위해 train set에만 적용하고 test set에는 적용하지 않는다.
#  이미지에 기하학적 변화를 주거나 확대/축소하고 회전을 시킨다.
#  training set의 이미지를 변화시켜 overfit이 발생하는 것을 방지한다.
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/3images/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4816 images belonging to 3 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/3images/validation',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 598 images belonging to 3 classes.


In [4]:
model = tf.keras.models.Sequential()

In [5]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))  

In [6]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [7]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) # input_shape는 맨 처음 레이어에만 입력하므로 두번째 레이어부터는 제거한다.
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
model.add(tf.keras.layers.Flatten())

In [9]:
model.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [10]:
model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

In [11]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
model.fit(x = training_set, validation_data = test_set, epochs = 25)
print(training_set.class_indices)

Epoch 1/25
151/151 [==============================] - 2233s 14s/step - loss: 0.4878 - accuracy: 0.8077 - val_loss: 0.3205 - val_accuracy: 0.8746
Epoch 2/25
151/151 [==============================] - 803s 5s/step - loss: 0.3085 - accuracy: 0.8887 - val_loss: 0.2866 - val_accuracy: 0.8980
Epoch 3/25
151/151 [==============================] - 815s 5s/step - loss: 0.2858 - accuracy: 0.8943 - val_loss: 0.2505 - val_accuracy: 0.9047
Epoch 4/25
151/151 [==============================] - 802s 5s/step - loss: 0.2631 - accuracy: 0.9041 - val_loss: 0.2984 - val_accuracy: 0.8930
Epoch 5/25
151/151 [==============================] - 813s 5s/step - loss: 0.2385 - accuracy: 0.9128 - val_loss: 0.3749 - val_accuracy: 0.8829
Epoch 6/25
151/151 [==============================] - 798s 5s/step - loss: 0.2313 - accuracy: 0.9186 - val_loss: 0.2272 - val_accuracy: 0.9114
Epoch 7/25
151/151 [==============================] - 792s 5s/step - loss: 0.2171 - accuracy: 0.9203 - val_loss: 0.3159 - val_accuracy: 0.88

In [33]:
# model.save("tmc_cv_model.h5")

In [32]:
import numpy as np
from keras.preprocessing import image
result = []
for i in range(1, 16) :
  test_image = image.load_img('/content/drive/MyDrive/3images/test/test'+str(i)+'.jpg', target_size = (64, 64))
  test_image = image.img_to_array(test_image)  # PIL 형식의 이미지를 배열로 바꿔준다.
  test_image = np.expand_dims(test_image, axis = 0)  
  # 우리가 만든 모델은 배치사이즈가 32이다. 그래서 예측할때도 같은 배치로 해줘야한다.
  # 따라서 추가 배치를 해준다.
  result.append(np.argmax(model.predict(test_image)[0]))

print(result)

[0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


In [39]:
label = [0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1]
eq_cnt = 0
for i in range(len(label)):
  if result[i] == label[i]:
      eq_cnt+=1

print(eq_cnt/len(label)*100, '%')



66.66666666666666 %
